# DataFrames: Read and Write Data
     
Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  In this example we read and write data with the popular CSV and Parquet formats, and discuss best practices when using these formats.

In [1]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/0eEsIA0O1iE?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [2]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

Client Scheduler: inproc://192.168.1.235/78170/1 Dashboard: http://192.168.1.235:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [3]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [4]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [5]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [6]:
!head data/2000-01-01.csv

timestamp,id,name,x,y
2000-01-01 00:00:00,1020,Patricia,0.6857367738614941,0.673375942737479
2000-01-01 00:00:01,942,Quinn,0.9394041434089988,-0.06105721772738715
2000-01-01 00:00:02,1030,Ursula,-0.957167660708786,-0.3913359530607774
2000-01-01 00:00:03,983,Hannah,0.7907075196397082,-0.5061055159133385
2000-01-01 00:00:04,951,Bob,-0.4657153507911771,-0.1119312640727208
2000-01-01 00:00:05,1012,Ursula,0.9539102105834556,0.22420778108228756
2000-01-01 00:00:06,1004,Dan,0.08101500745582157,0.06503018863013188
2000-01-01 00:00:07,1048,Quinn,0.1487762554708012,-0.8608164541967294
2000-01-01 00:00:08,985,Laura,0.2996632750665147,0.10875850694059741


In [7]:
!head data/2000-01-30.csv

timestamp,id,name,x,y
2000-01-30 00:00:00,1016,Wendy,0.9009372938114111,0.15634034287662324
2000-01-30 00:00:01,1019,Edith,0.9156868846451791,-0.5799319659529119
2000-01-30 00:00:02,996,Bob,0.7625475882695762,0.6144544797664664
2000-01-30 00:00:03,1011,Zelda,-0.9516792843749473,-0.41862698005542365
2000-01-30 00:00:04,970,Bob,-0.8093789717766435,-0.6619554126012417
2000-01-30 00:00:05,932,Ray,-0.2847998469844577,-0.3710542358040383
2000-01-30 00:00:06,1034,George,-0.8585611821917789,0.9081575861625641
2000-01-30 00:00:07,969,Michael,0.7306838449060558,0.4903566032337856
2000-01-30 00:00:08,1061,Ray,-0.009446825106754986,0.7934936798341574


We can read one file with `pandas.read_csv` or many files with `dask.dataframe.read_csv`

In [8]:
import pandas as pd

df = pd.read_csv('data/2000-01-01.csv')
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1020,Patricia,0.685737,0.673376
1,2000-01-01 00:00:01,942,Quinn,0.939404,-0.061057
2,2000-01-01 00:00:02,1030,Ursula,-0.957168,-0.391336
3,2000-01-01 00:00:03,983,Hannah,0.790708,-0.506106
4,2000-01-01 00:00:04,951,Bob,-0.465715,-0.111931


In [9]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,object,int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [10]:
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1020,Patricia,0.685737,0.673376
1,2000-01-01 00:00:01,942,Quinn,0.939404,-0.061057
2,2000-01-01 00:00:02,1030,Ursula,-0.957168,-0.391336
3,2000-01-01 00:00:03,983,Hannah,0.790708,-0.506106
4,2000-01-01 00:00:04,951,Bob,-0.465715,-0.111931


## Tuning read_csv

The Pandas `read_csv` function has *many* options to help you parse files.  The Dask version uses the Pandas function internally, and so supports many of the same options.  You can use the `?` operator to see the full documentation string.

In [12]:
pd.read_csv?

In [13]:
dd.read_csv?

In this case we use the `parse_dates` keyword to parse the timestamp column to be a datetime.  This will make things more efficient in the future.  Notice that the dtype of the timestamp column has changed from `object` to `datetime64[ns]`.

In [14]:
df = dd.read_csv('data/2000-*-*.csv', parse_dates=['timestamp'])
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


## Do a simple computation

Whenever we operate on our dataframe we read through all of our CSV data so that we don't fill up RAM.  This is very efficient for memory use, but reading through all of the CSV files every time can be slow.

In [15]:
%time df.groupby('name').x.mean().compute()

CPU times: user 3.9 s, sys: 967 ms, total: 4.87 s
Wall time: 2.97 s


name
Alice       0.004357
Bob         0.000484
Charlie    -0.000829
Dan         0.001384
Edith      -0.002651
Frank      -0.000353
George     -0.003070
Hannah      0.000484
Ingrid     -0.000803
Jerry      -0.002016
Kevin       0.001486
Laura       0.000192
Michael     0.002355
Norbert     0.001713
Oliver     -0.000125
Patricia   -0.001325
Quinn      -0.003635
Ray         0.000795
Sarah      -0.001590
Tim        -0.000061
Ursula     -0.002392
Victor     -0.000921
Wendy      -0.000435
Xavier     -0.002267
Yvonne      0.005296
Zelda       0.000587
Name: x, dtype: float64

## Write to Parquet

Instead, we'll store our data in Parquet, a format that is more efficient for computers to read and write.

In [17]:
df.to_parquet('data/2000-01.parquet', engine='pyarrow')

In [18]:
!ls data/2000-01.parquet/

_common_metadata part.14.parquet  part.21.parquet  part.29.parquet
_metadata        part.15.parquet  part.22.parquet  part.3.parquet
part.0.parquet   part.16.parquet  part.23.parquet  part.4.parquet
part.1.parquet   part.17.parquet  part.24.parquet  part.5.parquet
part.10.parquet  part.18.parquet  part.25.parquet  part.6.parquet
part.11.parquet  part.19.parquet  part.26.parquet  part.7.parquet
part.12.parquet  part.2.parquet   part.27.parquet  part.8.parquet
part.13.parquet  part.20.parquet  part.28.parquet  part.9.parquet


## Read from Parquet

In [19]:
df = dd.read_parquet('data/2000-01.parquet', engine='pyarrow')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [20]:
%time df.groupby('name').x.mean().compute()

CPU times: user 1.18 s, sys: 474 ms, total: 1.66 s
Wall time: 1.01 s


name
Alice       0.004357
Bob         0.000484
Charlie    -0.000829
Dan         0.001384
Edith      -0.002651
Frank      -0.000353
George     -0.003070
Hannah      0.000484
Ingrid     -0.000803
Jerry      -0.002016
Kevin       0.001486
Laura       0.000192
Michael     0.002355
Norbert     0.001713
Oliver     -0.000125
Patricia   -0.001325
Quinn      -0.003635
Ray         0.000795
Sarah      -0.001590
Tim        -0.000061
Ursula     -0.002392
Victor     -0.000921
Wendy      -0.000435
Xavier     -0.002267
Yvonne      0.005296
Zelda       0.000587
Name: x, dtype: float64

## Select only the columns that you plan to use

Parquet is a column-store, which means that it can efficiently pull out only a few columns from your dataset.  This is good because it helps to avoid unnecessary data loading.

In [21]:
%%time
df = dd.read_parquet('data/2000-01.parquet', columns=['name', 'x'], engine='pyarrow')
df.groupby('name').x.mean().compute()

CPU times: user 983 ms, sys: 244 ms, total: 1.23 s
Wall time: 799 ms


name
Alice       0.004357
Bob         0.000484
Charlie    -0.000829
Dan         0.001384
Edith      -0.002651
Frank      -0.000353
George     -0.003070
Hannah      0.000484
Ingrid     -0.000803
Jerry      -0.002016
Kevin       0.001486
Laura       0.000192
Michael     0.002355
Norbert     0.001713
Oliver     -0.000125
Patricia   -0.001325
Quinn      -0.003635
Ray         0.000795
Sarah      -0.001590
Tim        -0.000061
Ursula     -0.002392
Victor     -0.000921
Wendy      -0.000435
Xavier     -0.002267
Yvonne      0.005296
Zelda       0.000587
Name: x, dtype: float64

Here the difference is not that large, but with larger datasets this can save a great deal of time.

## Learn more

http://dask.pydata.org/en/latest/dataframe-create.html